# Intro to MLOps using ZenML

## 🌍 Overview

This repository is a minimalistic MLOps project intended as a starting point to learn how to put ML workflows in production. It features: 

- A very simple training pipeline that loads the a dataset and trains a model.

Within this notebook we will show you how simple it is to switch where your code runs and where your data is stored. You will also learn how all the metadata of your run is stored and accessible through ZenML.

Follow along this notebook to understand how you can use ZenML to productionalize your ML workflows!

## Run on Colab

You can use Google Colab to see ZenML in action, no installation
required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/quickstart.ipynb)

# 👶 Step 0. Install Requirements

Let's install ZenML to get started.

In [ ]:
!pip install "zenml[server]" pyarrow datasets transformers transformers[torch] torch sentencepiece

In [ ]:
from zenml.environment import Environment

# In case we are in a google colab, clone all additional relevant files
if Environment.in_google_colab():
    # Pull required modules from this example
    !git clone -b main https://github.com/zenml-io/zenml
    !cp -r zenml/examples/quickstart/* .
    !rm -rf zenml

In [ ]:
# Restart Kernel to ensure all libraries are properly loaded
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)


Please wait for the installation to complete before running subsequent cells. At
the end of the installation, the notebook kernel will automatically restart.

## ☁️ Step 1: Connect to your ZenML Server
To run this quickstart you need to connect to a ZenML Server. You can deploy it [yourself](https://docs.zenml.io/getting-started/deploying-zenml) or try it out for free, no credit-card required in our [ZenML Pro managed service](https://zenml.io/pro).

In [ ]:
zenml_server_url = "INSERT_YOUR_SERVER_URL_HERE"  # in the form "https://URL_TO_SERVER"

!zenml connect --url $zenml_server_url

In [ ]:
# Initialize ZenML and set the default stack
!zenml init

!zenml stack set default

Default stack in this case means the code will run on the machine that is running this notebook and all output data will be stored there as well.

In [ ]:
# Do the imports at the top
from typing_extensions import Annotated

import random
import torch
import pandas as pd
from zenml import step, pipeline, Model, get_step_context
from zenml.client import Client
from zenml.logger import get_logger

from datasets import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
)
from steps.model_trainer import T5_Model
from zenml.config import ResourceSettings, DockerSettings

from typing import Optional, List

from zenml import pipeline

from steps import load_data, tokenize_data, train_model, evaluate_model, test_random_sentences

from zenml.logger import get_logger

logger = get_logger(__name__)

# Initialize the ZenML client to fetch objects from the ZenML Server
client = Client()

## 🥇 Step 2: Run your first pipeline

We'll start off by importing our data and training a simple nlp model. In this quickstart we'll be working with a small dataset of sentences in old english paired with more modern formulations. The task is a text-to-text transformation.

When you're getting started with a machine learning problem you'll want to break down your code into distinct functions that load your data, bring it into the correct shape and finally produce a model. H#
Here is our first function.

In [ ]:
@step
def load_data() -> Annotated[Dataset, "raw_dataset"]:
    """Load and prepare the dataset."""

    def read_data(file_path):
        inputs = []
        targets = []

        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                old, modern = line.strip().split("|")
                inputs.append(f"Translate Old English to Modern English: {old}")
                targets.append(modern)

        return {"input": inputs, "target": targets}

    # Assuming your file is named 'translations.txt'
    data = read_data("translations.txt")
    return Dataset.from_dict(data)

The whole function is decorated with the ZenML-`@step` decorator. Once this step is added to a pipeline, ZenML will automatically version, track, and cache the data that is produced by this function as an `artifact`. This enables you to 
reproduce your data at any point in the future, even if the original data source
changes or disappears. 

Note the typing of the function outputs. These are not only good practice, but also
help ZenML store and load your data appropriately. By using `Annotated` type hint in the output of the
step, we are also naming our outputs. This will make
it possible to access it by name later on.

ZenML is built in a way that allows you to experiment with your data and build
your pipelines one step at a time.  If you want to call this function to see how it
works, you can just call it directly. Here we take a look at the first few rows
of your training dataset.

In [ ]:
dataset = load_data()
dataset[0]

Everything looks as we'd expect and the values are all in the right format 🥳.

We're now at the point where can bring this step (and some others) together into a single
pipeline. To do this simply plug multiple steps together through their inputs and outputs.
Then just add the `@pipeline` decorator to the function that connects the steps.

In [ ]:
@pipeline()
def english_translation_pipeline(
    model_type: T5_Model,
    num_train_epochs: int,
    per_device_train_batch_size: int,
    gradient_accumulation_steps: int,
):
    """Define a pipeline that connects the steps."""
    dataset = load_data()
    tokenized_dataset = tokenize_data(dataset)
    model, tokenizer = train_model(tokenized_dataset, model_type, num_train_epochs, per_device_train_batch_size, gradient_accumulation_steps)
    evaluate_model(model, tokenized_dataset)
    test_random_sentences(model, tokenizer)

We're ready to run the pipeline now, which we can do just as with the step - by calling the
pipeline function itself:

In [ ]:
pipeline_run = configured_english_translation_pipeline(model_type="t5-small")

As you can see the pipeline has run succesfully. Lets check this out by following the Dashboard URL that you can find in the logs above. 

We can also fetch the pipeline from the server and view the results directly in the notebook:

In [ ]:
client = Client()
run = client.get_pipeline("english_translation_pipeline").last_run
print(run.name)

We can also access the trained model directly by accessing the run. The cool thing here is, this direct access is gonna be available not just now, within this notebook session, but at any later point when you or your colleaugues might need it.

In [ ]:
# load the model object
model = run.steps["train_model"].outputs["model"].load()
tokenizer = run.steps["train_model"].outputs["tokenizer"].load()

With these in hand we can now play around with the model directly and try out some examples ourselves:

In [ ]:
test_text = "I do desire we may be better strangers"

input_ids = tokenizer(
    test_text,
    return_tensors="pt",
    max_length=128,
    truncation=True,
    padding="max_length",
).input_ids

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=128,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded_output)

## Lets recap what we've done so far

1) We have created a pipeline that takes in a dataset and trains a small Model on it
2) This pipeline is broken down in such a way that we can easily iterate on the individual parts without breaking the whole

As expected, the performance of this model is not good. To train a model that can solve our task well, we wopuld have to train a larger model. For this, we'll need to move onto the next section.

# ⌚ Step 3: Scale it up in the cloud

The model we have trained on our local machine is proof that our pipeline works. However, we want to train a much more powerful model. For this we'll need to scale onto more powerful machines.

So lets take this to the next level and run the pipeline in the environment of your choice. In ZenML we use the word "stack" to describe the environment for a pipline run. A stack consists of different components, however you'll only really need to care about the compute and data storage for this example.

For you to be able to try this step, you will need to have acess to some cloud compute and cloud storage somewhere (aws, gcp, azure, etc...). ZenML wrapps around all the major cloud providers and orchestration tools and lets you easily plug your code into them.

To do this lets head over to the Stack section of your ZenML Dashboard. Here you'll be able to either connect to an existing or deploy a new environment. Choose on of the options presented to you there and come back when you have a stack ready to go. 

<img src=".assets/stack_creation.png" width="50%" alt="Pipelines Overview">

In [ ]:
from zenml.client import Client

# Set the name of your stack here
stack_name = "YOU_STACK_NAME_GOES_HERE"

Client().activate_stack(stack_name)

ZenML runs your pipeline on cloud compute by dockerizing the pipeline. In order to do so we need to install the appropriate requirements for tyour cloud provider and set the prepared parent image for quicker run times.

## GCP

In [ ]:
!zenml integration install gcp -y

from zenml.integrations.gcp.flavors.vertex_orchestrator_flavor import VertexOrchestratorSettings


configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "resources": ResourceSettings(memory="32GB")
        "orchestrator.vertex": VertexOrchestratorSettings(node_selector_constraint=("cloud.google.com/gke-accelerator", "NVIDIA_TESLA_P4"))
    }
)

## AWS

In [ ]:
!zenml integration install aws s3 -y

from zenml.integrations.aws.flavors.sagemaker_orchestrator_flavor import SagemakerOrchestratorSettings

configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "resources": ResourceSettings(memory="32GB"),
        "orchestrator.sagemaker": SagemakerOrchestratorSettings(instance_type="ml.p2.xlarge")
    }
)

## Azure

In [ ]:
!zenml integration install azure -y

from zenml.integrations.skypilot.flavors.skypilot_orchestrator_base_vm_config import SkypilotBaseOrchestratorSettings

configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "orchestrator.sagemaker": SkypilotBaseOrchestratorSettings(accelerators='V100', memory="32+", cpus="8+")
    }
)

## Ready to launch

We now have configured zenml to use your very own cloud infrastructure for the next pipeline run, lets see this in action by running the pipeline again on the smaller t5 model (`t5_small`).

Note: The whole process may take a bit longer the first time around, as your pipeline code needs to be built into docker containers to be run in the orchestration environment of your stack. Any consecutive run of the pipeline, even with different parameters set, will not take as long again thanks to docker caching.

In [ ]:
pipeline_run = configured_english_translation_pipeline(model_type="t5-small")

In [ ]:
...
...
...

In [ ]:
...

In [ ]:
...

In [ ]:
... summary ...

## Congratulations!

You're a legit MLOps engineer now! You have created a training pipeline and you
have deployed it into a production-ready environment with the compute of your 
choice. You also have gotten a hang of the ZenML Dashboard.

## Further exploration

This was just the tip of the iceberg of what ZenML can do; check out the [**docs**](https://docs.zenml.io/) to learn more
about the capabilities of ZenML. For example, you might want to:

- [Deploy ZenML](https://docs.zenml.io/user-guide/production-guide/connect-deployed-zenml) to collaborate with your colleagues.
- Run the same pipeline on a [cloud MLOps stack in production](https://docs.zenml.io/user-guide/production-guide/cloud-stack).
- Track your metrics in an experiment tracker like [MLflow](https://docs.zenml.io/stacks-and-components/component-guide/experiment-trackers/mlflow).

## What next?

* If you have questions or feedback... join our [**Slack Community**](https://zenml.io/slack) and become part of the ZenML family!
* If you want to quickly get started with ZenML, check out [ZenML Pro](https://zenml.io/pro).